# Image Classification Training

This notebook trains a CNN model for image classification using TensorFlow/Keras.

## Setup
First, let's import the necessary libraries and set up our environment.


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
from PIL import Image
import glob

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")


## Data Loading and Preprocessing

Load images from the data/images directory. Make sure your images are organized in subdirectories by class.


In [ ]:
def load_images_from_directory(directory_path, img_size=(224, 224)):
    """
    Load images from subdirectories and return X (images) and y (labels)
    """
    images = []
    labels = []
    class_names = []
    
    # Get all subdirectories (classes)
    subdirs = [d for d in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, d))]
    subdirs.sort()  # Sort for consistent ordering
    
    print(f"Found {len(subdirs)} classes: {subdirs}")
    
    for class_idx, class_name in enumerate(subdirs):
        class_names.append(class_name)
        class_path = os.path.join(directory_path, class_name)
        
        # Get all image files in this class directory
        image_files = glob.glob(os.path.join(class_path, "*.jpg")) + \
                     glob.glob(os.path.join(class_path, "*.jpeg")) + \
                     glob.glob(os.path.join(class_path, "*.png"))
        
        print(f"Loading {len(image_files)} images from class '{class_name}'")
        
        for image_file in image_files:
            try:
                # Load and preprocess image
                img = Image.open(image_file)
                img = img.convert('RGB')  # Convert to RGB
                img = img.resize(img_size)  # Resize to target size
                img_array = np.array(img) / 255.0  # Normalize to [0,1]
                
                images.append(img_array)
                labels.append(class_idx)
            except Exception as e:
                print(f"Error loading image {image_file}: {e}")
    
    return np.array(images), np.array(labels), class_names

# Load the data
data_path = 'data/images'
X, y, class_names = load_images_from_directory(data_path)

print(f"\nDataset shape: {X.shape}")
print(f"Number of classes: {len(class_names)}")
print(f"Class names: {class_names}")
print(f"Labels shape: {y.shape}")
print(f"Unique labels: {np.unique(y)}")


In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")

# Convert labels to categorical (one-hot encoding)
num_classes = len(class_names)
y_train_categorical = keras.utils.to_categorical(y_train, num_classes)
y_test_categorical = keras.utils.to_categorical(y_test, num_classes)

print(f"Training labels shape: {y_train_categorical.shape}")
print(f"Test labels shape: {y_test_categorical.shape}")

# Display some sample images
plt.figure(figsize=(12, 8))
for i in range(12):
    plt.subplot(3, 4, i + 1)
    plt.imshow(X_train[i])
    plt.title(f"Class: {class_names[y_train[i]]}")
    plt.axis('off')
plt.tight_layout()
plt.show()


## Model Architecture

Create a CNN model for image classification.


In [ ]:
def create_model(input_shape, num_classes):
    """
    Create a CNN model for image classification
    """
    model = keras.Sequential([
        # First convolutional block
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        
        # Second convolutional block
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Third convolutional block
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Fourth convolutional block
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Flatten and dense layers
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

# Create the model
input_shape = X_train.shape[1:]  # (224, 224, 3)
model = create_model(input_shape, num_classes)

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Display model summary
model.summary()
